In [9]:
from os import listdir

In [10]:
# ARGUMENTS

# main_path = 'C:\Users\Rafa\Downloads\'
# path_to_images = '/home/guscerra@GU.GU.SE/aics-project/celeb_images/CelebA-HQ-img/'
# captions_dir = r"C:\Users\Rafa\Downloads\text\celeba-caption"

dataset = 'celeb_small_res'
main_path = '/Users/evelsve/repos/cap'
path_to_images = main_path + f'/data/{dataset}'
captions_dir = f'/Users/evelsve/repos/cap/data/captions'


predictions_file = "predictions_m5_sketches.json"

In [11]:
# Checking whether the generated sketches match the original ones in the data set 

import os
import json 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def fiter_out_stop_words(sentence):
    tokenized_sent = word_tokenize(sentence)
    tokenized_sent = [word.lower().strip() for word in tokenized_sent if word.isalpha()]
    stop_words = set(stopwords.words('English'))
    filtered_caption = [word for word in tokenized_sent if word not in stop_words]
    return filtered_caption

In [14]:
def get_caps(predictions_dict, captions_dir):

    predicted_captions = list()
    real_captions = list()

    #''' Stop words filtered out '''
    for i in predictions_dict.keys():
        # dropping <start> and <end> tags and turning the caption list into a string
        predicted_caption = (' ').join(predictions_dict[i][1:-1])

        # dropping the stop words from PREDICTED
        predicted_caption = fiter_out_stop_words(predicted_caption)
        
        predicted_captions.append(predicted_caption)

        # loading the original captions
        # real_captions_path = captions_dir + '\\' + i + '.txt'
        real_caption_path = captions_dir + '/' + i + '.txt'
        with open(real_caption_path, 'r', encoding='utf-8') as f:
            real_caption = f.readlines()

        # pre-processing the predicted captions
        real_caption = [caption.lower().strip().replace('.', '').replace(',','') for caption in real_caption]
        # dropping the stop words from PREDICTED
        real_caption = [fiter_out_stop_words(caption) for caption in real_caption]
        real_captions.append(real_caption)
        
    return predicted_captions, real_captions
    
    
def reformat_to_bigrams(data):
    reformatted_to_bigrams = list()
    for sentence in data:
        bigrammed = list()
        for i, word in enumerate(sentence[:-1]):
            bigram = (sentence[i], sentence[i+1])
            bigrammed.append(bigram)
        reformatted_to_bigrams.append(bigrammed)
    return reformatted_to_bigrams


def evaluate(predicted, actual, f):
    predicted = reformat_to_bigrams(predicted)
    
    new_actual = list()
    for item in actual:
        item = reformat_to_bigrams(item)
        new_actual.append(item)
    
    actual = new_actual
    
    scores = dict()

    for i1, predicted_caption in enumerate(predicted):
        
        scores[i1] = dict()
        
        for i2, real_caption in enumerate(actual[i1]):
            if predicted_caption == real_caption:
                f.write(f'\nComplete match at image {i1} caption number {i2}')
            tp = 0
            for tupler in real_caption:
                if tupler in predicted_caption:
                    tp += 1
            try:
                score = tp/len(real_caption)*100
            except:
                score = 0

            scores[i1][i2] = score
            
    return scores


def average_evaluations(scores, actual, maximize=True):
    new_scores = list()
    for img_id, per_caption_scores in scores.items():
        per_caption_scores = per_caption_scores.values()
        score_list = list(per_caption_scores)
        if maximize:
            biggest = max(score_list)
            new_scores.append(biggest)
        else:
            avg = sum(score_list)/10
            new_scores.append(avg)
    return sum(new_scores)/len(actual)


def pipeline(main_path):
    predictions_dir = main_path + '/predictions'
    with open(f'{main_path}/feature_eval_results.txt', mode='w') as f:
        for predictions_file in listdir(main_path + '/predictions'):
            if predictions_file[-5:] == '.json' and predictions_file != 'augmented.json':
                f.write(f'\n--- {predictions_file} ---\n')
                predictions_path = open(predictions_dir + '/' + predictions_file, 'rb')
                predictions_dict = json.load(predictions_path)
                predicted, actual = get_caps(predictions_dict, captions_dir)
                scores = evaluate(predicted, actual, f)
                f.write(f'\nResult: {average_evaluations(scores, actual)}\n')
    

In [15]:
pipeline(main_path)